IPSD
- Muhammad Iqbal (103102400020)
- Gracella Mangalik (103102400011)
- Khalid Muhammad (103102400033)
- Ivana Gabby Lauretta (103102400037)
- Izam Rosiawan (103102400049)

In [ ]:
from google.colab import drive
drive.mount('/content/Drive')

Drive already mounted at /content/Drive; to attempt to forcibly remount, call drive.mount("/content/Drive", force_remount=True).


In [ ]:
import os
import sqlite3
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Membuat connection ke database CoffeeShop
DB = sqlite3.connect('/content/Drive/MyDrive/IPSD/CoffeeShop_Dataset.db')

# Membuat cursor object untuk memanggil SQL statement
C = DB.cursor()

In [ ]:
# Load data from tables into pandas DataFrames
sales_reciepts = pd.read_sql_query("SELECT * FROM \"sales reciepts\"", DB)
product = pd.read_sql_query("SELECT * FROM product", DB)
sales_outlet = pd.read_sql_query("SELECT * FROM \"sales outlet\"", DB)
pastry_inventory = pd.read_sql_query("SELECT * FROM \"pastry inventory\"", DB)
customer = pd.read_sql_query('SELECT * FROM customer', DB)

## KHALID MUHAMMAD


### 1. Berapa rata-rata total pengeluaran per transaksi pelanggan?

In [ ]:
AVGSpend = pd.read_sql("""
SELECT AVG(TOTAL) AS AVGSpending
FROM (
    SELECT transaction_id, SUM(line_item_amount) AS TOTAL
    FROM "sales reciepts"
    GROUP BY transaction_id
);
""", DB)

AVGSpend

,AVGSpending
0,55.587901


Subquery menghitung SUM(line_item_amount) per transaksi.

Query utama mengambil AVG() dari total tiap transaksi.

Hasilnya: rata-rata pengeluaran pelanggan per transaksi.

### 2. Bagaimana distribusi pelanggan berdasarkan jenis kelamin (Gender Distribution)?

In [ ]:
GDist = customer['gender'].value_counts().reset_index()
GDist.columns = ['Gender', 'Jumlah_Pelanggan']
GDist

,Gender,Jumlah_Pelanggan
0,F,977
1,M,726
2,N,543


### Menunjukkan proporsi pelanggan pria dan wanita di database.

### Bisa dipakai untuk analisis demografi, misalnya:

- Apakah mayoritas pelanggan adalah perempuan?

- Apakah strategi promosi perlu disesuaikan berdasarkan gender dominan?

## IVANA GABBY LAURETA

### 1. Produk kopi atau pastry mana yang memiliki total penjualan tertinggi dan bagaimana hubungan antara promo dengan peningkatan penjualan produk tersebut?

In [ ]:
df_produk = pd.merge(sales_reciepts, product, on='product_id')
penjualan_per_produk = df_produk.groupby(['product', 'promo_yn'])['quantity'].sum().reset_index()
penjualan_per_produk = penjualan_per_produk.sort_values(by='quantity', ascending=False)
produk_terlaris = penjualan_per_produk.iloc[0]

print(f"Produk terlaris adalah {produk_terlaris['product']} dengan promo = {produk_terlaris['promo_yn']}")

Produk terlaris adalah Earl Grey Rg dengan promo = N


Tujuan dari soal ini yaitu untuk mengetahui produk mana yang paling banyak terjual dan melihat apakah diskon berpengaruh terhadap jumlah penjualannya. Langkahnya yaitu menggabungkan 2 tabel (`sales_reciepts` dan `product`) menggunakan `product_id` sebagai penghubung antara 2 tabel berikut. `groupby(['product', 'promo_yn'`]) untuk mengelompokkan data berdasarkan nama produk dan status promonya, lalu item terjualnya dijumlahkan menggunakan `.sum()` dan hasilnya dikembalikan dalam bentuk DataFrame menggunakan `reset_index()` sehingga tabel menampilkan total produk terjual untuk setiap produk dan dipisah berdasarkan apakah dijual dengan promo atau tidak. Produk diurutkan berdasarkan jumlah penjualan per produk dari yang tertinggi sampai terendah. `iloc[0]` mengambil data baris pertama di index 0 dari hasil urutan produk terlaris.


### 2. Produk mana yang memiliki tingkat waste tertinggi di setiap outlet?


In [ ]:
df_waste = pd.merge(pastry_inventory, product, on='product_id')
df_waste = pd.merge(df_waste, sales_outlet, on='sales_outlet_id')

df_waste['waste'] = df_waste['waste'].astype(str).str.replace('%', '', regex=False).astype(float)

total_waste_per_produk = df_waste.groupby('product')['waste'].sum().reset_index()
total_waste_per_produk = total_waste_per_produk.sort_values(by='waste', ascending=False)
produk_waste_tertinggi = total_waste_per_produk.iloc[0]

print(f"Produk dengan waste tertinggi adalah {produk_waste_tertinggi['product']} dengan total waste {produk_waste_tertinggi['waste']}")

Produk dengan waste tertinggi adalah Ginger Scone dengan total waste 2140.0


Tujuan dari soal ini yaitu untuk mengetahui produk mana yang paling banyak terbuang sehingga dievaluasi stok dan produksinya. Dengan cara menggabungkan 3 tabel (`pastry_inventory`, `product`, dan `sales_outlet`). Ubah `waste` string menjadi float (angka), lalu menghitung total waste per produk dengan mengelompokkan data per product menggunakan `groupby('product')`, menjumlahkan waste dari semua outlet menggunakan `sum()`, dan mengurutkan produk tersebut dari yang paling besar hingga yang paling kecil menggunakan `sort_values()`. `iloc[0]` digunakan untuk mengambil produk dengan waste tertinggi sehingga didapatkan output produk dengan waste tertinggi adalah Ginger Scone dengan total waste 2140.0


## IZAM ROSIAWAN

### 1. Mencari produk yang paling sering dibeli oleh pelanggan perempuan.

Saya mengambil data produk dari tabel penjualan, tapi hanya untuk pelanggan yang gender-nya perempuan.

Setelah itu, saya kelompokkan berdasarkan nama produk, lalu diurutkan dari yang paling banyak dibeli ke yang paling sedikit.

In [ ]:
df = pd.read_sql_query("""
    SELECT p.product
    FROM 'sales reciepts' sr
    JOIN customer c ON sr.customer_id = c.customer_id
    JOIN product p ON sr.product_id = p.product_id
    WHERE c.gender = 'F'
    GROUP BY p.product
    ORDER BY COUNT(*) DESC """, DB)

df

,product
0,Our Old Time Diner Blend Sm
1,Our Old Time Diner Blend Rg
2,Traditional Blend Chai Rg
3,Sustainably Grown Organic Lg
4,Brazilian Sm
...,...
75,Lemon Grass
76,Primo Espresso Roast
77,Spicy Eye Opener Chai
78,Peppermint


### 2. Mencari produk yang paling sering dipesan oleh pelanggan dari generasi Gen Z.

Saya mengambil data produk dari tabel penjualan, lalu memilih hanya transaksi dari Gen Z.

Setelah itu, data dikelompokkan berdasarkan nama produk dan diurutkan dari jumlah terbanyak.

In [ ]:
df = pd.read_sql_query("""
    SELECT p.product
    FROM 'sales reciepts' sr
    JOIN customer c ON sr.customer_id = c.customer_id
    JOIN generations g ON c.birth_year = g.birth_year
    JOIN product p ON sr.product_id = p.product_id
    WHERE g.generation = 'Gen Z'
    GROUP BY p.product
    ORDER BY COUNT(*) DESC """, DB)

df

,product
0,Peppermint Rg
1,Jamaican Coffee River Sm
2,Morning Sunrise Chai Lg
3,Brazilian Sm
4,Spicy Eye Opener Chai Rg
...,...
74,Ethiopia
75,Espresso Roast
76,Columbian Medium Roast
77,Brazilian - Organic


## MUHAMMAD IQBAL

### 1. berapa total penjualan tiap outlet?

In [ ]:
soi = pd.read_sql_query("""
                        SELECT
                            sales_outlet_id,
                            SUM(line_item_amount) AS total_penjualan
                        FROM
                            "sales reciepts"
                        GROUP BY
                            sales_outlet_id;
                        """, DB)
soi

,sales_outlet_id,total_penjualan
0,3,77213.23
1,5,76894.47
2,8,79528.25


### 2. Siapa saja 5 customer pertama berdasarkan tanggal daftar?

In [ ]:
top5 = pd.read_sql_query("""
                            SELECT
                                customer_id,
                                "customer_first-name" AS nama_pelanggan,
                                customer_since
                            FROM
                                customer
                            ORDER BY
                                customer_since ASC
                            LIMIT 5;
                        """, DB)
top5

,customer_id,nama_pelanggan,customer_since
0,8294,Charlotte,2017-01-03
1,8483,Samuel,2017-01-03
2,1,Kelly Key,2017-01-04
3,301,Alika Rivas,2017-01-04
4,5500,Dana,2017-01-04


## GRACELLA MANGALIK


### 1. Apakah harga produk yang diberi promo memang lebih rendah dibandingkan harga normalnya (non-promo)?

In [ ]:
# Tambahkan kolom generation dari birth_year
def get_generation(year):
    if 1946 <= year <= 1964:
        return 'Baby Boomers'
    elif 1965 <= year <= 1980:
        return 'Gen X'
    elif 1981 <= year <= 1996:
        return 'Millennial'
    elif 1997 <= year <= 2012:
        return 'Gen Z'
    else:
        return 'Gen Alpha'

customer['generation'] = customer['birth_year'].apply(get_generation)

# Hitung lama jadi pelanggan (pakai tahun sekarang 2025)
customer['lama_langganan'] = 2025 - pd.to_datetime(customer['customer_since']).dt.year

# Rata-rata lama langganan per generasi
loyal_gen = (
    customer.groupby('generation')['lama_langganan']
    .mean()
    .reset_index()
    .sort_values(by='lama_langganan', ascending=False)
)

print(loyal_gen)

     generation  lama_langganan
0  Baby Boomers        7.810964
1         Gen X        7.490368
3    Millennial        7.171910
2         Gen Z        6.421875


In [ ]:
# untuk bersihin kolom harga biar bisa dihitung (hapus tanda $ terus ubah jadi angka)
product['harga_bersih'] = product['current_retail_price'].str.replace('$', '').astype(float)

#untuk menghitung rata-rata harga buat produk promo (Y) dan non-promo (N)
rata_harga = product.groupby('promo_yn')['harga_bersih'].mean().reset_index()

# untuk nampilin hasil perbandingan harga rata-rata
print("💰 Perbandingan rata-rata harga produk promo vs non-promo:")
display(rata_harga)

# untuk menyimpan rata-rata harga promo dan harga normal ke variabel biar bisa dibandingin
harga_promo = rata_harga.loc[rata_harga['promo_yn'] == 'Y', 'harga_bersih'].values[0]
harga_normal = rata_harga.loc[rata_harga['promo_yn'] == 'N', 'harga_bersih'].values[0]

# untuk bandingin harga promo sama harga normal dan nampilin kesimpulannya
if harga_promo < harga_normal:
    print("\nKesimpulan: Produk yang diberi promo harganya terbukti lebih murah dari harga normalnya.")
elif harga_promo > harga_normal:
    print("\nKesimpulan: Produk promo justru punya harga lebih tinggi dari harga normalnya.")
else:
    print("\nKesimpulan: Harga produk promo dan harga normal ternyata sama aja.")


💰 Perbandingan rata-rata harga produk promo vs non-promo:


,promo_yn,harga_bersih
0,N,6.722976
1,Y,3.612500



Kesimpulan: Produk yang diberi promo harganya terbukti lebih murah dari harga normalnya.


### 2. Bulan apa yang menghasilkan penjualan tertinggi sepanjang tahun?


In [ ]:
# Ensure 'transaction_date' is in datetime format
sales_reciepts['transaction_date'] = pd.to_datetime(sales_reciepts['transaction_date'])

# Extract month name from 'transaction_date'
sales_reciepts['bulan_nama'] = sales_reciepts['transaction_date'].dt.strftime('%B')

# untuk menghitung total penjualan tiap bulan dari kolom line_item_amount
total_per_bulan = sales_reciepts.groupby('bulan_nama')['line_item_amount'].sum().reset_index()

# untuk mencariari bulan dengan total penjualan paling tinggi
bulan_tertinggi = total_per_bulan.loc[total_per_bulan['line_item_amount'].idxmax()]

# untuk nampilin hasil tabel
print("📅 Bulan dengan penjualan tertinggi sepanjang tahun:")
display(total_per_bulan)

# untuk nampilin kesimpulan
print(f"\nKesimpulan: Bulan {bulan_tertinggi['bulan_nama']} memiliki total penjualan tertinggi sepanjang tahun yaitu sebesar {bulan_tertinggi['line_item_amount']:.2f}.")

📅 Bulan dengan penjualan tertinggi sepanjang tahun:


,bulan_nama,line_item_amount
0,April,233635.95



Kesimpulan: Bulan April memiliki total penjualan tertinggi sepanjang tahun yaitu sebesar 233635.95.
